In [9]:
#| default_exp sharedUtilities
#| default_cls_lvl 3

# Shared utilties

In [10]:
#| export
import xarray as xr
from glob import glob
from tqdm import tqdm 
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

In [12]:
#| export
def preprocess(ds):
    """
    Used data has overlapping time data since they contain one month before the actual year
    and one month after the actual year.
    The function selects only the year of interest without the overlapping time data.

    Args:
        ds (xr.Dataset): netCDF4 data

    Returns:
        xr.Dataset: data without time overlap
    """
    return ds.sel(time=str(ds.time[len(ds.time)//2].dt.year.data)).resample(time="1D").mean()


In [13]:
#| export
def read_netcdfs(files, dim, transform_func, transform_calendar=None, cftime = True):
    """Reads multiples netcdfs files"""
    def process_one_path(path):
        try:
            if transform_calendar is not None:
                calendar = False
            else:
                calendar = True
            with xr.open_dataset(path, decode_times = calendar, use_cftime = cftime) as ds:
                if transform_calendar is not None:
                    ds[dim].attrs['calendar'] = transform_calendar
                    ds = xr.decode_cf(ds, use_cftime = cftime)
                if transform_func is not None:
                    ds = transform_func(ds)
                ds.load()
                return ds
        except Exception as e:
            print(path)
            print(e)
            return
    paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in tqdm(paths)]
    datasets = [x for x in datasets if x is not None]
    combined = xr.concat(datasets, dim)
    return combined

In [14]:
#| export
def transform_calendar(ds,
                       timedim="time",
                       calendarname="proleptic_gregorin"):
    """Transforms calendar of time index in xarray dataset"""
    ds[timedim].attrs['calendar'] = calendarname
    return ds

In [15]:
#| export

def plot_loss_and_acc(
    log_dir, loss_ylim=(0.0, 0.9), acc_ylim=(0.7, 1.0), save_loss=None, save_acc=None
):

    metrics = pd.read_csv(f"{log_dir}/metrics.csv")

    aggreg_metrics = []
    agg_col = "epoch"
    for i, dfg in metrics.groupby(agg_col):
        agg = dict(dfg.mean())
        agg[agg_col] = i
        aggreg_metrics.append(agg)

    df_metrics = pd.DataFrame(aggreg_metrics)
    df_metrics[["train_loss_epoch", "val_loss"]].plot(
        grid=True, legend=True, xlabel="Epoch", ylabel="Loss"
    )

    plt.ylim(loss_ylim)
    if save_loss is not None:
        plt.savefig(save_loss)

    if save_acc is not None:
        plt.savefig(save_acc)